# MINST

In [1]:
import tensorflow as tf
import numpy as np

import tensorflow_datasets as tfds

tf.__version__

'2.0.0'

## Pre-processing data

In [3]:
(train_image, train_labels), (test_image, test_labels) = tf.keras.datasets.mnist.load_data(path='mnist.npz')

import matplotlib.pyplot as plt

plt.imshow(train_image[0])

In [4]:
# Reshape
train_image = train_image.reshape(train_image.shape[0], 784)
test_image = test_image.reshape(test_image.shape[0], 784)

# Normalize
train_image = train_image/255
test_image = test_image/255 

# One hot encoding
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_image.shape, train_image.dtype)

Using TensorFlow backend.


(60000, 784) float64


## Tensorflow model Neural Networks

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.InteractiveSession()

Instructions for updating:
non-resource variables are not supported in the long term


**Placeholders**

In [6]:
x  = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

**Initialize W and b**

In [7]:
weights = tf.Variable(tf.random_normal([784, 10], stddev = 0.01))
bias = tf.Variable(tf.random_normal([10], stddev = 0.01))
                   
init = tf.global_variables_initializer()
sess.run(init)
sess.run(weights).shape
                   
print('weights shape',sess.run(weights).shape)

weights shape (784, 10)


**Linear Model**

In [8]:
logits = tf.matmul(x, weights) + bias

**Activation function**

In [9]:
y = tf.nn.softmax(logits)

**Hyperparameters**

In [10]:
epochs = 1000
learning_rate = 0.5

**Loss and Optimizer**

The cost function and the optimizer are the most critical definitions.

In [11]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

### Training

In [12]:
for epoch in range(epochs):
    _, loss = sess.run([optimizer, cross_entropy], feed_dict={x:train_image[:50,:], y_:train_labels[:50,:]})
    if epoch % 100 == 0:
        print('epoch:', epoch,
            'cross entropy: ', loss)

epoch: 0 cross entropy:  2.281452
epoch: 100 cross entropy:  0.02119354
epoch: 200 cross entropy:  0.010373167
epoch: 300 cross entropy:  0.006871515
epoch: 400 cross entropy:  0.0051398217
epoch: 500 cross entropy:  0.004106617
epoch: 600 cross entropy:  0.0034200058
epoch: 700 cross entropy:  0.0029305804
epoch: 800 cross entropy:  0.0025640011
epoch: 900 cross entropy:  0.0022791591


**Accuracy**

In [13]:
# correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# acc = accuracy.eval(feed_dict={x:test_image[:50,:], y_:test_labels[:50,:]}) * 100
# print("The final accuracy for the simple ANN model is: {} % ".format(acc))

In [14]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for epoch in range(epochs):
    _, loss, acc = sess.run([optimizer, cross_entropy, accuracy], feed_dict={x:test_image[:50,:], y_:test_labels[:50,:]})
    if epoch % 100 == 0:
        print('epoch:', epoch,
            'cross entropy: ', loss,
             'accuracy:', acc)

epoch: 0 cross entropy:  0.95590174 accuracy: 0.72
epoch: 100 cross entropy:  0.01354075 accuracy: 1.0
epoch: 200 cross entropy:  0.007000536 accuracy: 1.0
epoch: 300 cross entropy:  0.004761392 accuracy: 1.0
epoch: 400 cross entropy:  0.0036195081 accuracy: 1.0
epoch: 500 cross entropy:  0.0029243326 accuracy: 1.0
epoch: 600 cross entropy:  0.0024556974 accuracy: 1.0
epoch: 700 cross entropy:  0.0021179593 accuracy: 1.0
epoch: 800 cross entropy:  0.0018627853 accuracy: 1.0
epoch: 900 cross entropy:  0.0016630995 accuracy: 1.0


In [15]:
sess.close()

## Tensorflow model CNN

In [16]:
sess = tf.InteractiveSession()

### Arquitecture

**Placeholders**

In [17]:
x  = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [18]:
x_image = tf.reshape(x, [-1,28,28,1])  
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

**Convolutional layer 1**

*Weights and bias (kernels)*

In [19]:
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))

*Conv2d*

In [20]:
convolve1 = tf.nn.conv2d(x_image, W_conv1, strides = [1, 1, 1, 1], padding = 'SAME') + b_conv1

**ReLu**

In [21]:
h_conv1 = tf.nn.relu(convolve1)

**MaxPooling**

In [22]:
conv1 = tf.nn.max_pool(h_conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
conv1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

**Convolutional layer 2**

*Weights and bias (kernels)*

In [23]:
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

*Conv2d*

In [24]:
convolve2 = tf.nn.conv2d(conv1, W_conv2, strides = [1, 1, 1, 1], padding = 'SAME') + b_conv2

**ReLu**

In [25]:
h_conv2 = tf.nn.relu(convolve2)

**MaxPooling**

In [26]:
conv2 = tf.nn.max_pool(h_conv2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
conv2

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>

**Flatten**

In [27]:
layer2_matrix = tf.reshape(conv2, [-1, 7 * 7 * 64])

**Fully Connected FC1**

*Weights and bias*

In [28]:
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev = 0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape = [1024]))

*logits*

In [29]:
fc1 = tf.matmul(layer2_matrix, W_fc1) + b_fc1

*activation function*

In [30]:
h_fc1 = tf.nn.relu(fc1)

*Dropout*

In [31]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)
layer_drop

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<tf.Tensor 'dropout/mul_1:0' shape=(?, 1024) dtype=float32>

**Fully Connected FC2**

*Weights and bias*

In [32]:
W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev = 0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape = [10]))

*logits*

In [33]:
fc=tf.matmul(layer_drop, W_fc2) + b_fc2

*SOFTMAX*

In [34]:
y_CNN= tf.nn.softmax(fc)
y_CNN

<tf.Tensor 'Softmax_1:0' shape=(?, 10) dtype=float32>

### Train CNN

In [35]:
epochs = 300
learning_rate = 1e-4

**Loss and Optimizer**

In [42]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_CNN), reduction_indices=[1]))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

**Accuracy**

In [43]:
correct_prediction = tf.equal(tf.argmax(y_CNN, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [44]:
sess.run(tf.global_variables_initializer())

In [45]:
for epoch in range(epochs):
    _, loss, acc = sess.run([optimizer, cross_entropy, accuracy], feed_dict={x: train_image[:50], y_: train_labels[:50],
                                                                              keep_prob: 1.0})
    if epoch % 100 == 0 :
        print('epoch:', epoch,
               'loss:', loss,
               'acc:', acc)

epoch: 0 loss: 7.7851105 acc: 0.08
epoch: 100 loss: 0.015191991 acc: 1.0
epoch: 200 loss: 0.00411716 acc: 1.0
